In [1]:
import re
import json
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
data_folder = './data'
main_dataset = 'compas-scores.csv'
#main_dataset2 = 'compas-scores-raw.csv'

In [5]:
df_init = pd.read_csv (data_folder+'/'+main_dataset)
df = df_init

In [6]:
df.shape

(11757, 47)

In [7]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,num_r_cases,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,num_vr_cases,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN,-1,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,NaN,13009779CF10A,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,1,NaN,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,NaN,13011511MM10A,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [8]:
columns_to_drop = ['last', 'first', 'vr_charge_degree', 'vr_case_number', 'num_vr_cases', 'c_days_from_compas', 'days_b_screening_arrest', 'juv_other_count', 'juv_misd_count', 'decile_score', 'juv_fel_count', 'vr_charge_desc', 'v_type_of_assessment', 'v_type_of_assessment', 'v_decile_score', 'v_score_text', 'type_of_assessment', 'decile_score.1', 'score_text']

df = df.drop(columns_to_drop, axis=1)
df.head()

,id,name,compas_screening_date,sex,dob,age,age_cat,race,priors_count,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_charge_degree,c_charge_desc,is_recid,num_r_cases,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,vr_offense_date,v_screening_date,screening_date
0,1,miguel hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,F,Aggravated Assault w/Firearm,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,2013-08-14,2013-08-14
1,2,michael ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,NaN,NaN,NaN,NaN,NaN,O,NaN,-1,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,2014-12-31,2014-12-31
2,3,kevon dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,F,Felony Battery w/Prior Convict,1,NaN,13009779CF10A,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,1,2013-07-05,2013-01-27,2013-01-27
3,4,ed philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,4,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,F,Possession of Cocaine,1,NaN,13011511MM10A,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN,2013-04-14,2013-04-14
4,5,marcu brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,1,NaN,NaN,13000570CF10A,2013-01-12,NaN,F,Possession of Cannabis,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN,2013-01-13,2013-01-13


### Base RF model

In [9]:
columns_to_drop = ['id', 'c_case_number', 'num_r_cases', 'r_case_number']

df = df.drop(columns_to_drop, axis=1)
df.head()

,name,compas_screening_date,sex,dob,age,age_cat,race,priors_count,c_jail_in,c_jail_out,c_offense_date,c_arrest_date,c_charge_degree,c_charge_desc,is_recid,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,vr_offense_date,v_screening_date,screening_date
0,miguel hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,2013-08-13 06:03:42,2013-08-14 05:41:20,2013-08-13,NaN,F,Aggravated Assault w/Firearm,0,O,NaN,NaN,NaN,NaN,NaN,0,NaN,2013-08-14,2013-08-14
1,michael ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,NaN,NaN,NaN,NaN,O,NaN,-1,O,NaN,NaN,NaN,NaN,NaN,0,NaN,2014-12-31,2014-12-31
2,kevon dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,2013-01-26 03:45:27,2013-02-05 05:36:53,2013-01-26,NaN,F,Felony Battery w/Prior Convict,1,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,1,2013-07-05,2013-01-27,2013-01-27
3,ed philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,4,2013-04-13 04:58:34,2013-04-14 07:02:04,2013-04-13,NaN,F,Possession of Cocaine,1,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN,2013-04-14,2013-04-14
4,marcu brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,1,NaN,NaN,2013-01-12,NaN,F,Possession of Cannabis,0,O,NaN,NaN,NaN,NaN,NaN,0,NaN,2013-01-13,2013-01-13


In [10]:
df = df[df['is_recid'] != -1]
df = df[df['is_violent_recid'] != -1]

In [11]:
df.shape

(11038, 25)

In [12]:
is_recid, is_violent_recid = df['is_recid'], df['is_violent_recid']

In [13]:
columns_to_drop = ['is_recid', 'is_violent_recid']

df = df.drop(columns_to_drop, axis=1)
df.head()

,name,compas_screening_date,sex,dob,age,age_cat,race,priors_count,c_jail_in,c_jail_out,c_offense_date,c_arrest_date,c_charge_degree,c_charge_desc,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,vr_offense_date,v_screening_date,screening_date
0,miguel hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,2013-08-13 06:03:42,2013-08-14 05:41:20,2013-08-13,NaN,F,Aggravated Assault w/Firearm,O,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-14,2013-08-14
2,kevon dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,2013-01-26 03:45:27,2013-02-05 05:36:53,2013-01-26,NaN,F,Felony Battery w/Prior Convict,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,2013-07-05,2013-01-27,2013-01-27
3,ed philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,4,2013-04-13 04:58:34,2013-04-14 07:02:04,2013-04-13,NaN,F,Possession of Cocaine,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,2013-04-14,2013-04-14
4,marcu brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,1,NaN,NaN,2013-01-12,NaN,F,Possession of Cannabis,O,NaN,NaN,NaN,NaN,NaN,NaN,2013-01-13,2013-01-13
5,bouthy pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,2,NaN,NaN,NaN,2013-01-09,F,arrest case no charge,O,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-26,2013-03-26


In [14]:
columns_to_drop_temp = ['age_cat', 'r_charge_degree' , 'r_days_from_arrest', 'name', 'c_arrest_date', 'compas_screening_date', 'c_charge_desc', 'r_charge_desc', 'dob', 'c_jail_in', 'c_jail_out', 'c_offense_date', 'r_offense_date', 'r_jail_in', 'r_jail_out', 'vr_offense_date', 'v_screening_date', 'screening_date']

df = df.drop(columns_to_drop_temp, axis=1)

In [15]:
df['sex'] = df['sex'].map({'Male':1, 'Female':0})

In [16]:
race_map = {'Other':0, 'Caucasian':1, 'African-American':2, 'Hispanic':3, 'Asian':4, 'Native American':5}
df['race'] = df['race'].map(race_map)

In [17]:
# age_cat_map = {'Less than 25':0, '25 - 45':1, 'Greater than 45':2}
# df['age_cat'] = df['age_cat'].map(age_cat_map)

In [18]:
charge_degree_map = {'F':0, 'O':1, 'M':2}

df['c_charge_degree'] = df['c_charge_degree'].map(charge_degree_map)


#df['r_charge_degree'] = df['r_charge_degree'].map(charge_degree_map)
#df['r_days_from_arrest'] = df['r_days_from_arrest'].fillna(0)

In [19]:
race_one_hot = pd.get_dummies(df['race'], prefix='race')

df = df.drop('race',axis = 1)
df = df.join(race_one_hot)

In [20]:
sex_one_hot = pd.get_dummies(df['sex'], prefix='sex')

df = df.drop('sex',axis = 1)
df = df.join(sex_one_hot)

In [21]:
df.head()

,age,priors_count,c_charge_degree,race_0,race_1,race_2,race_3,race_4,race_5,sex_0,sex_1
0,69,0,0,1,0,0,0,0,0,0,1
2,34,0,0,0,0,1,0,0,0,0,1
3,24,4,0,0,0,1,0,0,0,0,1
4,23,1,0,0,0,1,0,0,0,0,1
5,43,2,0,1,0,0,0,0,0,0,1


In [22]:
train, test, train_y, test_y = train_test_split(df,
                                                is_recid,
                                                test_size = 0.20,
                                                random_state = 42)

In [23]:
model =  RandomForestClassifier(n_estimators=10 ,
                               oob_score=True,
                               random_state=1)
model.fit(train, train_y)

/Users/akurochkin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/akurochkin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [24]:
pred = model.predict(test)

In [25]:
#print ("AUC-ROC (oob) = ", roc_auc_score(train_y, model.oob_prediction_))
print ("AUC-ROC (test) = ", roc_auc_score(test_y, pred))

AUC-ROC (test) =  0.5927476370014121
